In [52]:
#uses VectorAdd_#N(_noHR)(_S#noepsE_HRx#hrf)(_NIR, _IR) - v1.1x
#uses SItoSoE_#N_HRx#hrf_(NIL, IL)_(NOR, OR) - v1.1x

In [53]:
import sys
import math
import os 

In [54]:
import import_ipynb
from MatrixAdd_SI_V1_2 import MatrixAdd_SI

In [55]:
N = input("ENTER N The number of elements: ")
M = input("ENTER M: ")
PR = input("ENTER PR: ")
HRR = input("ENTER HRR(hardware reuse rate >=2 N<= ): ")
ENABLE_INPUT_REGISTERS = input("ENABLE_INPUT_REGISTERS(empty or 0 to cancel, 1 to enable)?: ")
ENABLE_INPUT_LATCH = input("ENABLE_INPUT_LATCH(empty or 0 to cancel, 1 to enable)?: ")
#IN_WIDTH>=1
IN_WIDTH = input("Enter elements width(bits): ")

ENTER N The number of elements: 10
ENTER M: 10
ENTER PR: 5
ENTER HRR(hardware reuse rate >=2 N<= ): 5
ENABLE_INPUT_REGISTERS(empty or 0 to cancel, 1 to enable)?: 
ENABLE_INPUT_LATCH(empty or 0 to cancel, 1 to enable)?: 
Enter elements width(bits): 


In [56]:
N=int(N)
M=int(M)

if not PR:
    PR=1
else :
    PR=int(PR)
    
if not IN_WIDTH:
    IN_WIDTH=10
else :
    IN_WIDTH=int(IN_WIDTH)

OUT_WIDTH=IN_WIDTH+1

if not HRR:
    HRR = 2
else:
    HRR = int(HRR)
    if HRR<2:
        HRR = 2
    if HRR>N:
        HRR = N
        
if not ENABLE_INPUT_REGISTERS:
    ENABLE_INPUT_REGISTERS = 0
else:
    ENABLE_INPUT_REGISTERS = int(ENABLE_INPUT_REGISTERS)
    
if not ENABLE_INPUT_LATCH:
    ENABLE_INPUT_LATCH = 0
else:
    ENABLE_INPUT_LATCH = int(ENABLE_INPUT_LATCH)
NAdd = math.ceil(N/HRR);
while math.ceil(N/NAdd) != HRR:
    HRR -= 1
print("HRR calculated value: {}".format(HRR))
NE = N - NAdd*(HRR-1)
while NE<=0:
    NE += NAdd
RS=math.ceil(M/PR)   
ar=RS*PR-M
ME = PR-ar-1
lgRS=math.ceil(math.log2(RS))
lghrr=math.ceil(math.log2(HRR))
parameters={
    "N":N,
    "M":M,
    "HRR":HRR,
    "PR":PR,
    "ENABLE_INPUT_REGISTERS":ENABLE_INPUT_REGISTERS,
    "ENABLE_INPUT_LATCH":ENABLE_INPUT_LATCH,
    "IN_WIDTH":IN_WIDTH
}
IN_DATA_LENGHT=PR*2*N*IN_WIDTH
OUT_DATA_LENGHT=PR*N*OUT_WIDTH
OUT_DATA_LENGHT_HR=PR*NAdd*OUT_WIDTH
OUT_DATA_LENGHT_NE= PR*NE*OUT_WIDTH
Print_To_File=1
Files_Location="MatrixAdd_GeneratedVerilogCodes/"
Is_Top=1
modules={}
GeneratedFileName=MatrixAdd_SI(parameters,Print_To_File,Files_Location,Is_Top,modules)
Files_Location+=GeneratedFileName+"/"
wrapperModuleName=GeneratedFileName+"_AXIStream"
of=open(Files_Location+wrapperModuleName+".v", 'w+')

HRR calculated value: 5


In [57]:
# in this cell we will use the generated file and instanciate it in our new AXI wrapper module.
print("`timescale 1ns / 1ps\n",file=of)
print("module "+wrapperModuleName,file=of)
print("#(",file=of)
print("parameter IN_DATA_LENGHT= {}, ".format(IN_DATA_LENGHT),file=of)
print("parameter OUT_DATA_LENGHT= {}, ".format(OUT_DATA_LENGHT),file=of)
print("parameter OUT_DATA_LENGHT_HR= {}".format(OUT_DATA_LENGHT_HR),file=of)
print(")( \n",file=of)
print("input aclk,".format(),file=of)
print("input aresetn,".format(),file=of)
print("input enable,".format(),file=of)
print("input [IN_DATA_LENGHT-1:0]s_axi_data,".format(),file=of)
print("input s_axi_valid,".format(),file=of)
print("input m_axi_ready,".format(),file=of)
print("output reg[OUT_DATA_LENGHT-1:0]m_axi_data,".format(),file=of)
print("output reg m_axi_valid,".format(),file=of)
print("output reg s_axi_ready".format(),file=of)
print(");\n".format(),file=of) 
print("reg inready;".format(),file=of)
print("reg [IN_DATA_LENGHT-1:0]indata;".format(),file=of)
print("reg [{}:{}] HR_Counter".format(lghrr-1,0),file=of)# to be filled
print("reg outready;".format(),file=of)#changed to register for always block.
print("wire HR_outready;".format(),file=of)
print("wire earlyoutready;".format(),file=of)
print("wire [OUT_DATA_LENGHT-1:0]outdata;".format(),file=of)
print("wire [OUT_DATA_LENGHT_HR-1:0]HR_outdata;".format(),file=of)

In [58]:
print("////////////////instancing wrapping module///////////////////".format(),file=of)
print("{} {}(".format(GeneratedFileName,GeneratedFileName+"_inst"),file=of)
print(".clk(aclk),".format(),file=of)
print(".reset(!aresetn),".format(),file=of)
print(".enable(enable),".format(),file=of)
print(".inReady(inready),".format(),file=of)
for j in range (PR):
    for i in range (N):
        print(".A{}V{}(indata[{}:{}]),".format( i, j, IN_DATA_LENGHT-((i+j*N)*(IN_WIDTH))-1 , IN_DATA_LENGHT-((i+j*N+1)*(IN_WIDTH)) ), file=of )
for j in range (PR):        
    for i in range (N):    
        print(".B{}V{}(indata[{}:{}]),".format( i, j, int(IN_DATA_LENGHT/2)-((i+j*N)*(IN_WIDTH))-1 , int(IN_DATA_LENGHT/2)-((i+j*N+1)*(IN_WIDTH)) ), file=of )
if ME==0:
    st1=".V0"
else:
    st1=".V0toV{}".format(ME)
if NE==1:
    print(st1+"S0outReady(HR_outready),", file=of)
else:
    print(st1+"S0toS{}outReady(HR_outready),".format(NE-1), file=of)
for j in range(PR):
    for i in range (NAdd):
        print(".S{}V{}(HR_outdata[{}:{}]),".format( i, j, OUT_DATA_LENGHT_HR-((i+j*NAdd)*OUT_WIDTH+1)  , OUT_DATA_LENGHT_HR-(i+j*NAdd+1)*OUT_WIDTH), file=of)
#print(".earlyOutReady(earlyoutready)".format(),file=of)
print(");".format(),file=of)

In [59]:
#in this cell we will add Axi stream compatability codes to wrapp the generated module in the previeous cell
print("/////////////////Main body/////////////".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  m_axi_data<=0;".format(),file=of)
print("  m_axi_valid<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  if(m_axi_ready==1 && m_axi_valid==1)begin".format(),file=of)
print("   m_axi_valid<=0;".format(),file=of)
print("  end".format(),file=of)
print("  else if(outready==1)begin".format(),file=of)
print("   m_axi_valid<=1;".format(),file=of)
print("   m_axi_data<=outdata;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)
print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  s_axi_ready<=1;".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  indata<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  inready<=0;".format(),file=of)
print("  if(s_axi_valid==1 && s_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=0;".format(),file=of)
print("   inready<=1;".format(),file=of)
print("   indata<= s_axi_data;".format(),file=of)
print("  end".format(),file=of)
print("  else if(m_axi_valid==1 && m_axi_ready==1)begin".format(),file=of)
print("   s_axi_ready<=1;".format(),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)

In [60]:
#in this part we add extra blocks to handle Hardware resuasability in SI mode

print("always @(posedge aclk)begin".format(),file=of)
print(" if(aresetn==0)begin".format(),file=of)
print("  HR_Counter<=0;".format(),file=of)
print(" end".format(),file=of)
print(" else begin".format(),file=of)
print("  if(HR_outready==1 && HR_Counter=={})begin".format(HRR-1),file=of)
print("   HR_Counter<=0;".format(),file=of)
print("   outdata[{}-HR_Counter*{}-1-:{}]<=HR_outdata[{}-:{}]".format(OUT_DATA_LENGHT, OUT_DATA_LENGHT_HR, OUT_DATA_LENGHT_NE, OUT_DATA_LENGHT_HR-1,OUT_DATA_LENGHT_NE),file=of)
print("  end".format(),file=of)
print("  else if(HR_outready)begin".format(),file=of)
print("   HR_Counter<=HR_Counter+1".format(),file=of)
print("   outdata[{}-HR_Counter*{}-1-:{}]<=HR_outdata".format(OUT_DATA_LENGHT,OUT_DATA_LENGHT_HR,OUT_DATA_LENGHT_HR),file=of)
print("  end".format(),file=of)
print(" end".format(),file=of)
print("end".format(),file=of)

print("always @(*)begin".format(),file=of)
print(" if(HR_outready==1 && HR_Counter=={})begin".format(HRR-1),file=of)
print("  outready<=1".format(),file=of)    
print(" end".format(),file=of)    
print(" else begin".format(),file=of)    
print("  outready<=0".format(),file=of)
print(" end".format(),file=of)    
print("end".format(),file=of)
print("\nendmodule",file=of)
of.close()